In [1]:
import spacy
from spacy import displacy
from lambeq.backend.drawing import draw
from lambeq.backend.grammar import Cup, Id, Ty, Word
from lambeq import AtomicType, IQPAnsatz, NumpyModel, BinaryCrossEntropyLoss, QuantumTrainer, SPSAOptimizer, Dataset
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

n, s = Ty('n'), Ty('s')
# nlp = spacy.load("ja_core_web_sm")
nlp = spacy.load("ja_ginza")

c:\Users\neoia\anaconda3\envs\lambeq_env2\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\neoia\anaconda3\envs\lambeq_env2\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
# データセット読み込み
def read_data(filename):
    labels, sentences = [], []
    with open(filename) as f:
        for line in f:
            t = int(line[0])
            # labels.append([[0, t],[0, 1-t]]) # 2×2行列の形で格納するならこっち
            labels.append([1-t, t]) # 1×2行列の形
            sentences.append(line[1:].strip())
            
    return np.array(labels), sentences

train_labels, train_sentences = read_data('C:/Users/neoia/研究/data/mc_train_data_jp.txt')
dev_labels, dev_sentences = read_data('C:/Users/neoia/研究/data/mc_dev_data_jp.txt')
test_labels, test_sentences = read_data('C:/Users/neoia/研究/data/mc_test_data_jp.txt')

In [3]:
# トークン数をカウントして最大・最小の文を探す関数
def print_max_min_tokens(sentences, name="dataset"):
    token_counts = [len(nlp(sent)) for sent in sentences]
    
    max_idx = np.argmax(token_counts)
    min_idx = np.argmin(token_counts)
    
    print(f"--- {name} ---")
    print(f"最大トークン数: {token_counts[max_idx]}, 文: {sentences[max_idx]}")
    print(f"最小トークン数: {token_counts[min_idx]}, 文: {sentences[min_idx]}\n")

# 各データセットで確認
print_max_min_tokens(train_sentences, "train")
print_max_min_tokens(dev_sentences, "dev")
print_max_min_tokens(test_sentences, "test")


--- train ---
最大トークン数: 11, 文: 熟練した男はソースを準備します。
最小トークン数: 7, 文: 女性は夕食を焼きます。

--- dev ---
最大トークン数: 11, 文: 熟練した人が申請を準備します。
最小トークン数: 7, 文: 男はソースを焼きます。

--- test ---
最大トークン数: 12, 文: 熟練した女性は夕食を作っています。
最小トークン数: 7, 文: 女性は食事を焼きます。

